# OpenAI Agent 

OpenAI 플랫폼을 사용하여 사용자를 대신해 행동(예: 컴퓨터 제어 등)을 수행할 수 있는 **에이전트(Agent)** 를 구축할 수 있습니다.  
Python용 **Agent SDK**를 사용하면 백엔드에서 이러한 에이전트의 **오케스트레이션(조율) 로직**을 만들 수 있습니다.

**OpenAI Agents SDK**는 **에이전트 기반 AI 애플리케이션**을 개발할 수 있게 해주는 도구입니다.  

**핵심 구성 요소**  

- **에이전트 (Agents)**:  
  LLM에 **지시문(instructions)** 과 **도구(tools)** 를 장착한 실행 단위  
- **핸드오프 (Handoffs)**:  
  특정 작업을 다른 에이전트에게 **위임**할 수 있도록 해줌  
- **가드레일 (Guardrails)**:  
  에이전트에게 전달되는 입력을 **검증**하는 장치

### 주요 기능 요약:

- **에이전트 루프 (Agent Loop)**:  
  도구 실행 → 결과 전달 → LLM 호출 → 반복 실행 → 완료까지 자동 처리

- **핸드오프 (Handoffs)**:  
  여러 에이전트 간의 **협업과 위임**을 유연하게 처리 가능

- **가드레일 (Guardrails)**:  
  에이전트 입력을 **사전 검사/검증**하여, 조건을 만족하지 않으면 **조기 종료 가능**

- **함수 기반 도구 (Function Tools)**:  
  Python 함수 하나를 **자동으로 에이전트 도구로 변환**,  
  **Pydantic 기반 스키마 자동 생성** 및 검증 포함

- **추적(Tracing)**:  
  워크플로우를 **시각화/디버깅/모니터링** 가능하며,  
  OpenAI의 평가/파인튜닝/디스틸레이션 툴과 통합 가능

In [1]:
# pip install openai-agents

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

### Simple Example

In [6]:
from agents import Agent, Runner

spanish_agent = Agent(
    name="한국어 agent",
    instructions="당신은 한국어만 할 수 있습니다.",
)

english_agent = Agent(
    name="영어 agent",
    instructions="당신은 영어만 할 수 있습니다.",
)

handoff_agent = Agent(
    name="분류 agent",
    instructions="요청에 사용된 언어에 따라 적절한 에이전트에게 넘겨주세요.",
    handoffs=[spanish_agent, english_agent],
)



result = await Runner.run(handoff_agent, input="폭싹 속았쑤까?")
print(result.final_output)

result = await Runner.run(handoff_agent, input="Are you happy?")
print(result.final_output)

"폭싹 속았쑤다"는 제주 방언을 제목으로 한 신작 드라마로 유명하죠. 혹시 이 드라마에 대해 더 궁금한 점이 있으신가요?
I'm just a computer program, so I don't have feelings like happiness. But I'm here to help you! How can I assist you today?


### Simple Agent

In [7]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="너는 친절한 도우미야.")

result = await Runner.run(
    starting_agent=agent,
    input="제주도 방언으로 혼저 옵소예가 무슨 뜻이야?"
)

result.final_output

'"혼저 옵소예"는 제주도 방언으로 "어서 오세요"라는 뜻이야. 환영의 인사를 표현하는 말이지. 제주도를 방문했을 때 이 표현을 들을 수 있을 거야.'

----------------------------
**`Runner.run_streamed()`는 비동기로 실행되며 `RunResultStreaming`을 반환합니다.**  
이 함수는 LLM을 **스트리밍 모드**로 호출하며, **LLM으로부터 받은 응답을 실시간으로 이벤트 형태로 스트리밍하여 전달**합니다.

In [9]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="짧은 이야기를 해줘"
)

async for event in response.stream_events():
    # print(event)
    if event.type == "raw_response_event":
        data = event.data
        if data.type == "response.output_text.delta":
            print(data.delta, end="", flush=True)

옛날 옛적 조용한 마을에 엘라라는 소녀가 살고 있었어. 엘라는 마을 외곽의 신비로운 숲을 매일같이 산책하며 다양한 동물들과 친구가 되었지. 어느 날, 숲 속에서 반짝이는 작은 돌을 발견했어. 그 돌은 마법을 품고 있었고, 엘라는 그것을 통해 동물들의 말을 이해할 수 있게 되었어.

엘라는 마법의 힘을 이용해 다친 동물을 도와주고, 마을과 숲의 평화를 유지했어. 시간이 지나면서 마을 사람들도 엘라의 특별한 능력을 알게 되었고, 그녀를 믿고 더욱 아끼게 되었지. 결국 엘라는 마을과 숲을 잇는 중요한 다리 같은 존재가 되었고 모두가 행복하게 살았다고 해.

------------------------
## 도구 (Tools)

**도구(Tools)** 는 에이전트가 **행동을 수행할 수 있도록 해주는 기능**입니다.  
예를 들어, **데이터를 가져오거나**, **코드를 실행하고**, **외부 API를 호출하거나**, 심지어 **컴퓨터를 사용하는 작업**까지도 가능합니다.

Agent SDK에서는 도구를 다음 **세 가지 유형**으로 구분합니다:  

### 1. **Hosted tools (호스팅 도구)**  
이 도구들은 **LLM 서버 내에서**, AI 모델과 함께 실행됩니다.  
OpenAI는 다음과 같은 호스팅 도구를 제공합니다:
- 검색 기반 정보 검색 (Retrieval)
- 웹 검색 (Web search)
- 컴퓨터 사용 (Computer use)

### 2. **Function calling (함수 호출 도구)**  
이 방식은 **Python 함수**를 도구로 등록하여 사용할 수 있게 해줍니다. 즉, 여러분이 직접 정의한 함수도 LLM이 도구처럼 호출할 수 있습니다.  

**Agents SDK**에서는 **모든 Python 함수를 도구(tool)** 로 사용할 수 있습니다. SDK는 이러한 도구를 자동으로 설정해줍니다:

1. **도구 이름**은 Python 함수의 이름에서 자동으로 생성됩니다.  

2. **도구 설명(description)** 은 함수의 **docstring**에서 가져옵니다.  

3. 함수 입력값의 **스키마(schema)** 는 함수의 **인자(arguments)** 로부터 자동 생성됩니다.

4. 각 입력값에 대한 **설명**도 함수의 docstring에서 추출됩니다.  

### 3. **Agents as tools (에이전트를 도구처럼 사용)**  
이 방식은 **하나의 에이전트를 도구로 사용**할 수 있게 해줍니다.  
즉, **핸드오프 없이도** 에이전트가 다른 에이전트를 호출할 수 있습니다.

-------------------------

`Runner.run()`을 호출하면, 최종 결과가 나올 때까지 루프가 실행됩니다.  이 루프의 동작 방식은 다음과 같습니다:

1. **LLM 호출:**  
   에이전트에 설정된 모델과 옵션, 그리고 대화 기록(message history)을 바탕으로 LLM을 호출.  

2. **LLM 응답 처리:**  
   모델이 응답을 반환하며, 여기에는 **툴 호출(tool calls)**이 포함될 수 있습니다.

3. **최종 출력이 있는 경우:**  
   응답에 **최종 출력(final output)**이 포함되어 있다면, 이를 반환하고 루프를 종료.

4. **핸드오프가 있는 경우:**  
   응답에 **다른 에이전트로의 핸드오프(handoff)**가 있으면, 현재 에이전트를 새로운 에이전트로 설정한 후 1단계부터 다시 시작.

5. **툴 호출 처리:**  
   툴 호출이 있을 경우, 해당 툴을 실행하고 결과 메시지를 기록한 다음 1단계로 되돌아갑니다.


**Final Output(최종 출력)** 은 루프에서 에이전트가 생성한 마지막 결과물입니다.

- 에이전트에 `output_type`이 설정되어 있다면:  
  → LLM이 해당 형식에 맞는 structured output(구조화된 출력)을 반환할 때 루프가 종료

- `output_type`이 설정되지 않은 경우:  
  → 툴 호출이나 핸드오프가 포함되지 않은 첫 번째 LLM 응답이 최종 출력으로 간주

In [10]:
# 함수 호출 example
from agents import function_tool

@function_tool
def multiply(x: float, y: float) -> float:
    """
    x 와 y 를 곱한다.
    """
    return x * y

agent = Agent(
    name="Assistant",
    instructions=(
        "당신은 유용한 도우미 입니다. 가능하면 제공된 도구를 사용하세요."
        "사용자 질의에 답하기 위해 당신의 지식에만 너무 의존하지 말고 도구를 사용하세요."
    ),
    model="gpt-4o-mini",
    tools=[multiply]
)

result = await Runner.run(agent, input="7.814 곱하기 103.892의 값은?")
print(result.final_output)

7.814 곱하기 103.892의 값은 811.812088입니다.


In [6]:
7.814 * 103.892

811.812088

## Guardrails (가드레일)

**가드레일(Guardrails)** 은 에이전트와 **병렬로 실행되며**, 사용자 입력에 대해 **검사와 유효성 검증**을 수행할 수 있게 해줍니다.

예를 들어, 아주 똑똑하지만 **느리고 비용이 많이 드는 모델**을 사용하는 에이전트를 악의적인 사용자가 수학 숙제를 대신 풀어달라고 요청하는 경우,
빠르고 저렴한 모델을 활용한 가드레일을 실행하여, 사용자의 입력이 악의적인 목적(예: 숙제 대행)인지 먼저 검사할 수 있습니다.  
가드레일이 의심스러운 입력을 감지하면 즉시 오류를 발생시켜 고비용 모델의 실행을 막고 시간과 비용을 절약할 수 있습니다.

In [11]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel

Model = "o3-mini"

# guardrail 에이전트의 출력 형식을 정의
# is_homework: 숙제인지 여부
# reasoning: 숙제로 판단한 이유 설명
class HomeworkOutput(BaseModel):
    is_homework: bool          # 이 입력이 숙제 질문인지 여부
    reasoning: str             # 판단 이유 설명

In [12]:
# 숙제 여부를 판단하는 guardrail 전용 에이전트 정의
guardrail_agent = Agent(
    name="Guardrail check",  # 에이전트 이름
    instructions="사용자가 숙제 질문을 하고 있는지 확인하세요.",  # LLM에게 줄 지시문
    model=Model,
    output_type=HomeworkOutput,  # 결과는 HomeworkOutput 형식으로 반환
)

# 수학 질문 전용 튜터 에이전트 정의
math_tutor_agent = Agent(
    name="Math Tutor",  # 에이전트 이름
    handoff_description="수학 질문을 위한 전문 에이전트",  # 다른 에이전트가 handoff할 때 참고하는 설명
    model=Model,
    instructions="당신은 수학 문제를 도와주는 튜터입니다. 각 단계의 이유를 설명하고 예시를 포함하세요.",  # LLM 지시문
)

# 역사 질문 전용 튜터 에이전트 정의
history_tutor_agent = Agent(
    name="History Tutor",  # 에이전트 이름
    handoff_description="역사 질문을 위한 전문 에이전트",  # 다른 에이전트가 handoff할 때 참고하는 설명
    model=Model,
    instructions="당신은 역사 질문을 도와주는 튜터입니다. 중요한 사건들과 그 맥락을 명확히 설명하세요.",  # LLM 지시문
)

# 입력이 '숙제 질문'인지 판단하는 guardrail 함수 정의
# triage_agent에서 InputGuardrail로 사용됨
async def homework_guardrail(ctx, agent, input_data):
    # guardrail_agent를 실행하여 숙제 여부 판단
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)

    # 결과를 HomeworkOutput 모델 형태로 변환
    final_output = result.final_output_as(HomeworkOutput)

    # GuardrailFunctionOutput을 반환
    # is_homework가 False이면 tripwire_triggered를 True로 설정하여 실행을 중단시킴
    return GuardrailFunctionOutput(
        output_info=final_output,               # guardrail 판단 결과
        tripwire_triggered=not final_output.is_homework,  # 숙제가 아니면 트리거 작동
    )

In [14]:
# 사용자 질문이 숙제인지 확인하고,
# 숙제라면 수학 튜터 또는 역사 튜터 에이전트로 전달(handoff)하는 판단 에이전트 정의
handoff_agent = Agent(
    name="Triage Agent",  # 에이전트 이름
    instructions="사용자의 숙제 질문을 기반으로 어떤 에이전트를 사용할지 결정하세요.",  
    handoffs=[history_tutor_agent, math_tutor_agent],  # 질문을 위임할 수 있는 대상 에이전트 목록
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),  # 입력값에 대해 사전 검사를 수행하는 가드레일
    ],
)

# triage_agent를 테스트하는 비동기 함수 정의
async def question(prompt):
    try:
        # 첫 번째 테스트: 역사 관련 숙제 질문
        result = await Runner.run(handoff_agent, prompt)
        print("✅ Output:", result.final_output)  # 출력 결과 출력
    except Exception as e:
        print("❌ Guardrail에 의해 입력이 거부되었습니다:", e)  # guardrail이 작동하면 예외 메시지 출력

# main 함수 실행 (Notebook 또는 async 환경에서 사용 가능)
prompt = "고구려의 첫번째 왕은 누구인가요?"
await question(prompt)

prompt = "사과와 감 중에 어느 것이 더 달아?"
await question(prompt)

✅ Output: 고구려의 첫 번째 왕은 주몽입니다. 주몽은 고구려를 창건한 인물로, 전설과 역사 속에서 중요한 역할을 합니다.

주몽에 관한 전설에서는 그의 출생과 모험, 그리고 결국 고구려 건국에 이르게 된 과정이 기록되어 있습니다. 그는 뛰어난 지략과 용맹으로 여러 지역의 부족들을 통합하고, 북쪽의 넓은 영토를 다스리는 강력한 국가를 세웠다고 전해집니다.

또한, 주몽은 동명왕이라는 이름으로도 알려져 있는데, 이 이름은 후대에 그를 기리며 사용된 칭호로 자리잡았습니다. 이러한 이야기는 《삼국유사》나 중국의 《사기》, 《후한서》 등 여러 고대 문헌에 등장하며, 고구려의 기원과 초기 역사에 큰 영향을 미쳤습니다.

즉, 고구려의 첫 번째 왕은 주몽(동명왕)으로, 그의 창건 이야기는 고구려의 초기 국가 형성과 그 왕조의 정신을 상징하는 중요한 역사적 서사로 남아 있습니다.
❌ Guardrail에 의해 입력이 거부되었습니다: Guardrail InputGuardrail triggered tripwire


### 구조화된 출력

In [15]:
from pydantic import BaseModel
from agents import Agent, Runner

# 캘린더 이벤트 데이터 구조 정의
class CalendarEvent(BaseModel):
    name: str              # 이벤트 이름
    date: str              # 이벤트 날짜
    participants: list[str]  # 참가자 목록

# 에이전트 정의: 텍스트에서 캘린더 이벤트 정보를 추출
agent = Agent(
    name="캘린더 추출기",  # 에이전트 이름
    instructions="텍스트에서 캘린더 이벤트를 추출하세요. "
                 "이벤트 이름, 날짜, 참가자 정보를 구조화된 데이터로 반환하세요.",
    output_type=CalendarEvent,  # 반환 데이터 형식
    model="gpt-4o",             # 사용할 LLM 모델
)

# 📌 실행 함수 정의
async def main():
    input_text = (
        "2025년 4월 10일에 '분기 전략 회의'라는 이름의 팀 미팅이 예정되어 있습니다. "
        "참가자는 오길동, 한철수, 김미미입니다."
    )

    result = await Runner.run(agent, input_text)  # 에이전트 실행
    print("추출된 캘린더 이벤트:")
    print(result.final_output)  # 결과 출력

# 📌 비동기 함수 실행
await main()

추출된 캘린더 이벤트:
name='분기 전략 회의' date='2025-04-10' participants=['오길동', '한철수', '김미미']
